In [1]:
# new notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import tzinfo, timedelta, datetime
import time
from collections import Counter
import json
import urllib
#from fuzzywuzzy import fuzz
#from googleplaces import GooglePlaces, types, lang
#import googleplaces
import re

In [2]:
parcel_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/SF_parcel_lu/sf_parcel.geojson'
#df_fire = 'D:/Study/insight/project/fire_risk_prediction_SF/data/SF_fire_data/fire-incidents.csv'
fire_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/Fire_Incidents_SF.csv'

In [3]:
df_fire = pd.read_csv(fire_file)

C:\Users\Ruoying\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (10,12,13,20,38,45,46,55,59) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_fire.shape

(538833, 64)

In [5]:
df_fire.head()

,Incident Number,Exposure Number,ID,Address,Incident Date,Call Number,Alarm DtTm,Arrival DtTm,Close DtTm,City,...,Detector Effectiveness,Detector Failure Reason,Automatic Extinguishing System Present,Automatic Extinguishing Sytem Type,Automatic Extinguishing Sytem Perfomance,Automatic Extinguishing Sytem Failure Reason,Number of Sprinkler Heads Operating,Supervisor District,neighborhood_district,point
0,8028304,0,80283040,150 Elsie St.,2008-04-01T00:00:00,80920257,2008-04-01T18:06:37,2008-04-01T18:15:19,2008-04-01T18:21:48,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,Bernal Heights,POINT (-122.41837339 37.74208979)
1,8028303,0,80283030,85 Turner Tr.,2008-04-01T00:00:00,80920256,2008-04-01T18:00:52,2008-04-01T18:06:30,2008-04-01T18:22:18,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,Potrero Hill,POINT (-122.39489 37.756291)
2,8028309,0,80283090,175 6th St.,2008-04-01T00:00:00,80920262,2008-04-01T18:42:06,2008-04-01T18:45:23,2008-04-01T18:53:25,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South of Market,POINT (-122.407468 37.78008)
3,8028314,0,80283140,633 Hayes St.,2008-04-01T00:00:00,80920268,2008-04-01T19:03:52,2008-04-01T19:08:39,2008-04-01T19:35:36,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,Hayes Valley,POINT (-122.42684908 37.77612642)
4,8028319,0,80283190,27th Av. / Cabrillo St.,2008-04-01T00:00:00,80920273,2008-04-01T19:16:12,2008-04-01T19:23:48,2008-04-01T19:28:49,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outer Richmond,POINT (-122.4863941 37.77428492)


In [6]:
df_fire.columns

Index(['Incident Number', 'Exposure Number', 'ID', 'Address', 'Incident Date',
       'Call Number', 'Alarm DtTm', 'Arrival DtTm', 'Close DtTm', 'City',
       'zipcode', 'Battalion', 'Station Area', 'Box', 'Suppression Units',
       'Suppression Personnel', 'EMS Units', 'EMS Personnel', 'Other Units',
       'Other Personnel', 'First Unit On Scene', 'Estimated Property Loss',
       'Estimated Contents Loss', 'Fire Fatalities', 'Fire Injuries',
       'Civilian Fatalities', 'Civilian Injuries', 'Number of Alarms',
       'Primary Situation', 'Mutual Aid', 'Action Taken Primary',
       'Action Taken Secondary', 'Action Taken Other',
       'Detector Alerted Occupants', 'Property Use', 'Area of Fire Origin',
       'Ignition Cause', 'Ignition Factor Primary',
       'Ignition Factor Secondary', 'Heat Source', 'Item First Ignited',
       'Human Factors Associated with Ignition', 'Structure Type',
       'Structure Status', 'Floor of Fire Origin', 'Fire Spread',
       'No Flame Spead'

In [7]:
df_fire['point'].head()

0    POINT (-122.41837339 37.74208979)
1         POINT (-122.39489 37.756291)
2         POINT (-122.407468 37.78008)
3    POINT (-122.42684908 37.77612642)
4     POINT (-122.4863941 37.77428492)
Name: point, dtype: object

In [8]:
# use WKT to generate geometric, needs:
# 1. string
# 2. not null

# create a test dataset that only have 1 column
test_fire = df_fire[['Incident Number','point']]
test_fire['point'] = test_fire['point'].astype(str)
test_fire = test_fire.dropna() # axis = 'point'

C:\Users\Ruoying\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
test_fire.shape

(538833, 2)

In [10]:
test_fire.head()

,Incident Number,point
0,8028304,POINT (-122.41837339 37.74208979)
1,8028303,POINT (-122.39489 37.756291)
2,8028309,POINT (-122.407468 37.78008)
3,8028314,POINT (-122.42684908 37.77612642)
4,8028319,POINT (-122.4863941 37.77428492)


In [11]:
# it seems that there is no null value, but instead, have a "nan" as a string
test_fire[test_fire['point'] == 'nan']

,Incident Number,point
86396,5091495,nan
88761,5099170,nan
91011,6002862,nan
248997,11074153,nan
259669,11084497,nan
...,...,...
528828,20058859,nan
529403,20061454,nan
531637,20036315,nan
536286,20086837,nan


In [12]:
test_fire = test_fire[test_fire['point'] != 'nan']

In [13]:
test_fire.shape

(538746, 2)

In [3]:
import sys

In [4]:
'geopandas' in sys.modules

False

In [5]:
!conda list

# packages in environment at C:\Users\Ruoying\anaconda3\envs\geoenv:
#
# Name                    Version                   Build  Channel
attrs                     20.2.0                     py_0  
backcall                  0.2.0                      py_0  
blas                      1.0                         mkl  
bzip2                     1.0.8                he774522_0  
ca-certificates           2020.7.22                     0  
certifi                   2020.6.20                py38_0  
cfitsio                   3.470                he774522_6  
click                     7.1.2                      py_0  
click-plugins             1.1.1                      py_0  
cligj                     0.5.0                    py38_0  
colorama                  0.4.3                      py_0  
curl                      7.67.0               h2a8f88b_0  
decorator                 4.4.2                      py_0  
expat                     2.2.9                h33f27b4_2  
fiona                 

In [6]:
jupyter kernelspec list

SyntaxError: invalid syntax (<ipython-input-6-68dcfb427789>, line 1)

it seems that geopandas have conflicts in dependency with existing environment
so need to create a virtual environment to run geopandas
https://medium.com/@sourav_raj/ultimate-easiest-way-to-install-geopandas-on-windows-add-to-jupyter-notebook-which-will-a4b11223f4f2

In [1]:
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'

In [15]:
df_parcel = gpd.read_file(parcel_file)
print(df_parcel.head())
#states = states.to_json()

  yrbuilt from_st total_uses     shape_area cie   landuse  mips mapblklot  \
0    1912     838          0  3864.06176213   0  RESIDENT     0   0026013   
1    1900     844       3471   3361.0253507   0    MIXRES  3471   0026014   
2    1900     848          0  3272.03319647   0  RESIDENT     0   0026015   
3    1940     860          0  4570.76766739   0  RESIDENT     0   0026016   
4    1938     870          0  5225.82373482   0  RESIDENT     0   0026017   

  objectid     shape_leng  ... resunits pdr street retail to_st block_num  \
0        1  327.786363395  ...        5   0    BAY      0   842      0026   
1        2  320.352521518  ...        1   0    BAY      0   844      0026   
2        3  319.046088472  ...        2   0    BAY      0   850      0026   
3        4  279.353001525  ...        2   0    BAY      0   862      0026   
4        5   294.35433855  ...        2   0    BAY      0   872      0026   

    blklot lot_num med                                           geometry 

In [16]:
# objectid is the key
df_parcel.columns

Index(['yrbuilt', 'from_st', 'total_uses', 'shape_area', 'cie', 'landuse',
       'mips', 'mapblklot', 'objectid', 'shape_leng', 'visitor', 'st_type',
       'bldgsqft', 'resunits', 'pdr', 'street', 'retail', 'to_st', 'block_num',
       'blklot', 'lot_num', 'med', 'geometry'],
      dtype='object')

In [17]:
test_parcel = df_parcel[['objectid', 'geometry']]

In [18]:
type(test_parcel)

geopandas.geodataframe.GeoDataFrame

In [19]:
import geojsonio

In [20]:
#geojsonio.display(df_parcel)

In [2]:
# create a gpd from df_fire using "point" as geometry
#df_fire['point'].head()
from shapely import wkt

ModuleNotFoundError: No module named 'shapely'

In [22]:
test_fire['point'] = test_fire['point'].apply(wkt.loads)

In [23]:
gdf_fire = gpd.GeoDataFrame(test_fire, geometry='point')

In [28]:
# make sure the two dataset have the same projection

#gdf_fire = gdf_fire.to_crs({'init': 'epsg:4326'})
gdf_fire.crs = test_parcel.crs

In [29]:
# spatial join
fire_sjoin = gpd.sjoin(gdf_fire,test_parcel,how = 'left', op='within')

In [30]:
fire_sjoin.head()

,Incident Number,point,index_right,objectid
0,8028304,POINT (-122.41837 37.74209),127696.0,127685
1,8028303,POINT (-122.39489 37.75629),NaN,NaN
2,8028309,POINT (-122.40747 37.78008),NaN,NaN
3,8028314,POINT (-122.42685 37.77613),13586.0,13557
4,8028319,POINT (-122.48639 37.77428),NaN,NaN


In [32]:
fire_sjoin_not_null = fire_sjoin.dropna()
fire_sjoin_not_null.shape

(149305, 4)

In [33]:
# of all the 528k fires, 149k are associated with parcel
# now groupby objectid and rejoin with parcel data

grouped = fire_sjoin_not_null.groupby('objectid').size()
df_group = grouped.to_frame().reset_index()
df_group.columns = ['objectid','fire_count']

In [34]:
df_group.head()

,objectid,fire_count
0,1,2
1,10,5
2,100,1
3,1000,4
4,100004,1


In [35]:
# first convert gpd to pd
parcel = pd.DataFrame(df_parcel.drop(columns='geometry'))

# then merge using objectid
merged_data = parcel.merge(df_group, on='objectid', how='outer')

In [36]:
merged_data.head()

,yrbuilt,from_st,total_uses,shape_area,cie,landuse,mips,mapblklot,objectid,shape_leng,...,resunits,pdr,street,retail,to_st,block_num,blklot,lot_num,med,fire_count
0,1912,838,0,3864.06176213,0,RESIDENT,0,0026013,1,327.786363395,...,5,0,BAY,0,842,0026,0026013,013,0,2.0
1,1900,844,3471,3361.0253507,0,MIXRES,3471,0026014,2,320.352521518,...,1,0,BAY,0,844,0026,0026014,014,0,NaN
2,1900,848,0,3272.03319647,0,RESIDENT,0,0026015,3,319.046088472,...,2,0,BAY,0,850,0026,0026015,015,0,4.0
3,1940,860,0,4570.76766739,0,RESIDENT,0,0026016,4,279.353001525,...,2,0,BAY,0,862,0026,0026016,016,0,NaN
4,1938,870,0,5225.82373482,0,RESIDENT,0,0026017,5,294.35433855,...,2,0,BAY,0,872,0026,0026017,017,0,NaN


In [37]:
merged_data.to_csv('df_alpha.csv',index=False)